# Showcase of fjsp-drl

This model requires python 3.6 and package versions in `requirements.txt`.
This model solves FJSP instances, which are a generalisation of JSSP.

In [1]:
import torch, json, PPO_model, copy, os, gym, time
import numpy as np
from env.load_data import nums_detec

# JSSP TO FJSP
First I need to convert JSSP to FJSP instance. Below is an example of JSSP instance in [Taillard's specification](http://jobshop.jjvh.nl/explanation.php)

```
6	6
1	3	6	7	3	6
8	5	10	10	10	4
5	4	8	9	1	7
5	5	5	3	8	9
9	3	5	4	3	1
3	3	9	10	4	1
3	1	2	4	6	5
2	3	5	6	1	4
3	4	6	1	2	5
2	1	3	4	5	6
3	2	5	6	1	4
2	4	6	1	5	3
```

Below is the same problem as [FJSP](https://people.idsia.ch/~monaldo/fjsp.html)

```
6   6   1   
6   1   3   1   1   1   3   1   2   6   1   4   7   1   6   3   1   5   6   
6   1   2   8   1   3   5   1   5   10  1   6   10  1   1   10  1   4   4   
6   1   3   5   1   4   4   1   6   8   1   1   9   1   2   1   1   5   7   
6   1   2   5   1   1   5   1   3   5   1   4   3   1   5   8   1   6   9   
6   1   3   9   1   2   3   1   5   5   1   6   4   1   1   3   1   4   1   
6   1   2   3   1   4   3   1   6   9   1   1   10  1   5   4   1   3   1   
```

Helper functions to convert JSSP to FJSP:

In [2]:
def parse_instance_taillard(filename):
    '''Parses instance written in Taillard specification: http://jobshop.jjvh.nl/explanation.php
    
      Args:
        filename - file containing the instance in Taillard specification

      Returns:
        number of jobs,
        number of machines,
        the processor times for each operation,
        the order for visiting the machines
    '''

    with open(filename, 'r') as f:
        # parse number of jobs J and machines M
        J, M = map(int, f.readline().split())

        # Initialize two empty numpy arrays with dimensions J x M
        processor_times = np.empty((J, M), dtype=int)
        orders_of_machines = np.empty((J, M), dtype=int)
    
        # Read the next J lines containing processor times
        for i in range(J):
            processor_times[i] = list(map(int, f.readline().split()))
    
        # Read the next J lines containing orders of machines
        for i in range(J):
            orders_of_machines[i] = list(map(int, f.readline().split()))

        return J, M, processor_times, orders_of_machines

def jssp_taillard_to_fjsp(filename: str) -> str:
    '''Transforms JSSP instance in Taillard's specification to FJSP instance
       and stores it in a temporary file
    
      Args:
        filename - name of the file with JSSP instance in Taillard's specification
        
      Returns:
        string - filename of the equivalent FJSP instance 
    '''
    # parse JSSP Taillard instance
    J, M, processor_times, orders_of_machines = parse_instance_taillard(filename)
    
    # convert JSSP to FJSP
    with open("/tmp/fjsp_" + filename.split("/")[-1], 'w') as f:
        # write number of jobs, number of machines, and jobs/machines (which is always 1 for JSSP)
        f.write(str(J) + "   " + str(M) + "   1\n")
        
        # each line is a job
        for i in range(J):
            # each line starts with the number of operations in a job
            number_of_operations = len(processor_times[i])
            f.write(str(number_of_operations) + "  ")
            
            # print the operation as a tuple (number of available machines, current machine, processing time)
            for j in range(number_of_operations):
                f.write(" 1   " + str(orders_of_machines[i][j]) + "   " + str(processor_times[i][j]) + "  ")
                
            f.write('\n')
            
    return "/tmp/fjsp_" + filename.split("/")[-1]
    

# Model

## Model setup

Setup torch, load the configuration of the experiment prepare the model and load its parameters from the checkpoint

In [3]:
# setup torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device.type=='cuda':
    torch.cuda.set_device(device)
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')
    
# load configuration of the experiment
with open("./config.json", 'r') as load_f:
    load_dict = json.load(load_f)
    
model_paras = load_dict['model_paras']
model_paras["actor_in_dim"] = model_paras["out_size_ma"] * 2 + model_paras["out_size_ope"] * 2
model_paras["critic_in_dim"] = model_paras["out_size_ma"] + model_paras["out_size_ope"]
model_paras['device'] = device
env_paras = load_dict['env_paras']
env_paras['device'] = device
env_paras['batch_size'] = 1

# create model and its parameters from the checkpoint
memories = PPO_model.Memory()
model_CKPT = torch.load('./model/save_10_5.pt')
model = PPO_model.PPO(model_paras, load_dict['train_paras'])
model.policy.load_state_dict(model_CKPT)
model.policy_old.load_state_dict(model_CKPT)

<All keys matched successfully>

In [4]:
def solve_instance(instance_file, model, memories, flag_sample=False):
    '''Solves FJSP instance using given model and memories
    
      Args:
          instance_file - name of the file with the FJSP instance
          model - model used to solve the instance
          memories - model's memories
    
      Returns:
          makespan and time it took to solve the instance
    '''
    with open(instance_file, 'r') as file_object:
        # load the parameters of the instance
        line = file_object.readlines()
        ins_num_jobs, ins_num_mas, _ = nums_detec(line)
        env_paras["num_jobs"] = ins_num_jobs
        env_paras["num_mas"] = ins_num_mas
        
        # create env and get states and completion signal
        env = gym.make('fjsp-v0', case=[instance_file], env_paras=env_paras, data_source='file')
        state = env.state
        dones = env.done_batch
        done = False  # Unfinished at the beginning
        
        # perform scheduling
        start = time.time()
        while not done:
            with torch.no_grad():
                actions = model.policy_old.act(state, memories, dones, flag_sample=flag_sample, flag_train=False)
            state, rewards, dones = env.step(actions)  # environment transit
            done = dones.all()
        run_time = time.time() - start  # The time taken to solve this environment (instance)

        gantt_result = env.validate_gantt()[0]
        if not gantt_result:
            raise Exception("Scheduling error")

        return copy.deepcopy(env.makespan_batch), run_time

## FJSP instances from the paper

Run the experiment on some instances from this paper.

In [5]:
TEST_FILES_PATH = "data_test/Public/"
test_files = sorted(os.listdir(TEST_FILES_PATH))
for file in test_files[:10]:
    makespan, _ = solve_instance(TEST_FILES_PATH + file, model, memories)
    print("Makespan of", file, ":", makespan.item())

Makespan of Mk01.fjs : 43.0
Makespan of Mk02.fjs : 38.0
Makespan of Mk03.fjs : 204.0
Makespan of Mk04.fjs : 81.0
Makespan of Mk05.fjs : 192.0
Makespan of Mk06.fjs : 99.0
Makespan of Mk07.fjs : 221.0
Makespan of Mk08.fjs : 529.0
Makespan of Mk09.fjs : 339.0
Makespan of Mk10.fjs : 264.0


## Run the model on JSSP benchmarks

Run the experiment only on the first 20 JSSP benchmarks due to extensive time needed to run the model.

In [6]:
def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../benchmarks/jssp"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

In [9]:
instances = get_all_instances_in_taillard_specification()
for instance in get_all_instances_in_taillard_specification()[:20]:
    fjsp_instance = jssp_taillard_to_fjsp(instance)
    makespan, run_time = solve_instance(fjsp_instance, model, memories)
    print(f"Makespan of {instance.split('/')[-1]}: {makespan.item()}, time: {run_time}")

Makespan of yn03.txt: 1073.0, time: 6.0707879066467285
Makespan of yn02.txt: 1043.0, time: 5.524256467819214
Makespan of yn01.txt: 1032.0, time: 5.595107316970825
Makespan of yn04.txt: 1247.0, time: 5.291898250579834
Makespan of swv14.txt: 3955.0, time: 9.079474210739136
Makespan of swv18.txt: 2852.0, time: 8.770984888076782
Makespan of swv20.txt: 2823.0, time: 8.306881666183472
Makespan of swv17.txt: 2794.0, time: 8.133539915084839
Makespan of swv19.txt: 3009.0, time: 8.139398574829102
Makespan of swv08.txt: 2401.0, time: 3.6731536388397217
Makespan of swv02.txt: 2098.0, time: 2.172487735748291
Makespan of swv05.txt: 1842.0, time: 2.149639129638672
Makespan of swv12.txt: 4036.0, time: 8.047986268997192
Makespan of swv01.txt: 1971.0, time: 2.0727322101593018
Makespan of swv06.txt: 2100.0, time: 3.44944167137146
Makespan of swv11.txt: 3961.0, time: 9.089773416519165
Makespan of swv13.txt: 4316.0, time: 9.019090175628662
Makespan of swv03.txt: 1919.0, time: 2.7358503341674805
Makespan of

#  Run the model on FJSP benchmarks

In [14]:
def get_all_fjsp_instances():
    '''Lists all FJSP instances'''
    matching_files = []
    root_dir = "../../../benchmarks/fjsp"
    target_string = ".fjs"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

In [17]:
for instance in sorted(get_all_fjsp_instances()):
    try:
        makespan, run_time = solve_instance(instance, model, memories)
        print(f"Makespan of {instance.split('/')[-1]}: {makespan.item()}, time: {run_time}")
    except Exception as e:
        print(f"Failed to solve instance {instance}, error: {e}")

Makespan of Behnke1.fjs: 153.0, time: 0.5056495666503906
Makespan of Behnke10.fjs: 187.0, time: 1.1273655891418457
Makespan of Behnke11.fjs: 352.0, time: 3.0933427810668945
Makespan of Behnke12.fjs: 372.0, time: 3.2885754108428955
Makespan of Behnke13.fjs: 391.0, time: 2.8087804317474365
Makespan of Behnke14.fjs: 368.0, time: 2.620962142944336
Makespan of Behnke15.fjs: 351.0, time: 2.618858575820923
Makespan of Behnke16.fjs: 662.0, time: 7.9500932693481445
Makespan of Behnke17.fjs: 652.0, time: 7.963798522949219
Makespan of Behnke18.fjs: 637.0, time: 7.989922046661377
Makespan of Behnke19.fjs: 671.0, time: 8.164292812347412
Makespan of Behnke2.fjs: 159.0, time: 0.5509364604949951
Makespan of Behnke20.fjs: 688.0, time: 7.8297364711761475
Makespan of Behnke21.fjs: 133.0, time: 0.5764336585998535
Makespan of Behnke22.fjs: 145.0, time: 0.5918543338775635
Makespan of Behnke23.fjs: 139.0, time: 0.6996376514434814
Makespan of Behnke24.fjs: 142.0, time: 0.6605157852172852
Makespan of Behnke25.

Makespan of HurinkSdata64.fjs: 1194.0, time: 1.3324766159057617
Makespan of HurinkSdata65.fjs: 1066.0, time: 1.2405250072479248
Makespan of HurinkSdata66.fjs: 1143.0, time: 1.1522178649902344
Makespan of HurinkSdata7.fjs: 724.0, time: 0.5656795501708984
Makespan of HurinkSdata8.fjs: 593.0, time: 0.5996212959289551
Makespan of HurinkSdata9.fjs: 926.0, time: 1.0229897499084473
Makespan of HurinkEdata1.fjs: 55.0, time: 0.4490354061126709
Makespan of HurinkEdata10.fjs: 969.0, time: 0.948420524597168
Makespan of HurinkEdata11.fjs: 969.0, time: 0.8718483448028564
Makespan of HurinkEdata12.fjs: 900.0, time: 0.7696502208709717
Makespan of HurinkEdata13.fjs: 937.0, time: 0.763725996017456
Makespan of HurinkEdata14.fjs: 1221.0, time: 1.0346429347991943
Makespan of HurinkEdata15.fjs: 1001.0, time: 1.0159623622894287
Makespan of HurinkEdata16.fjs: 1142.0, time: 1.0086474418640137
Makespan of HurinkEdata17.fjs: 1231.0, time: 1.0696258544921875
Makespan of HurinkEdata18.fjs: 1356.0, time: 1.19701814

Makespan of HurinkRdata61.fjs: 784.0, time: 1.3394887447357178
Makespan of HurinkRdata62.fjs: 894.0, time: 1.3924169540405273
Failed to solve instance ../../../benchmarks/fjsp/2_Hurink/2c_Hurink_rdata/HurinkRdata63.fjs, error: index 99 is out of bounds for dimension 0 with size 99
Makespan of HurinkRdata64.fjs: 819.0, time: 1.2644984722137451
Makespan of HurinkRdata65.fjs: 831.0, time: 1.2590863704681396
Makespan of HurinkRdata66.fjs: 916.0, time: 1.286787986755371
Makespan of HurinkRdata7.fjs: 586.0, time: 0.6088087558746338
Makespan of HurinkRdata8.fjs: 489.0, time: 0.6290493011474609
Makespan of HurinkRdata9.fjs: 815.0, time: 0.976815938949585
Makespan of HurinkVdata1.fjs: 47.0, time: 0.4571354389190674
Makespan of HurinkVdata10.fjs: 785.0, time: 0.9885344505310059
Makespan of HurinkVdata11.fjs: 806.0, time: 0.9212179183959961
Makespan of HurinkVdata12.fjs: 871.0, time: 0.9801938533782959
Makespan of HurinkVdata13.fjs: 843.0, time: 0.9085190296173096
Makespan of HurinkVdata14.fjs: 1

Makespan of Fattahi20.fjs: 1676.0, time: 0.5144381523132324
Makespan of Fattahi3.fjs: 255.0, time: 0.061751604080200195
Makespan of Fattahi4.fjs: 355.0, time: 0.06717109680175781
Makespan of Fattahi5.fjs: 150.0, time: 0.07487702369689941
Makespan of Fattahi6.fjs: 360.0, time: 0.12626385688781738
Makespan of Fattahi7.fjs: 407.0, time: 0.10459327697753906
Makespan of Fattahi8.fjs: 293.0, time: 0.1129753589630127
Makespan of Fattahi9.fjs: 220.0, time: 0.11526298522949219
